In [ ]:
# Loading the code required to run the code below
import os
from ultralytics import YOLO

In [ ]:
# Was crashing due to an unknown error, and I found this workaround online. Apparently it solves some library conflicts when running multiple instances of Intels OpenMP.
# I would recommend not running this section, as the code should ideally function without it
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
# Specify the path to my datasets dataset.yaml file 
data_yaml_path = "tiled_dataset/dataset.yaml"

# Specify the optimizer from the paper used as inspiration
chosen_optimizer = "AdamW"

# Specify how many epochs the model should run for
epochs = 100
# Specify which image size the model should rescale the input to
imgsz = 640

# Load the model
v11_model = YOLO("yolo11n.pt")

# Here I override the current optimizer with the optimizer from the paper, which i specified before
v11_model.overrides['optimizer'] = chosen_optimizer

results = v11_model.train(
    # Uses the dataset.yaml file to access the train/images and train/labels folders, as well as the validation folders
    data=data_yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    # Run the training on my GPU
    device=0,
    # Specify that the computer should use less CPU to load the data, since I was running into crashes
    workers=1,
    # The batch size determines how many images the model has to go through before it updates its weights
    batch=16,
    learning_rate=0.01,

    # Name of the folder in which the results and weights are saved
    project="runs_YOLO11",

    # Here I apply the augmentations from paper, which is discussed in more detail in the thesis
    degrees=10,           
    shear=15,
    hsv_v=0.25,

    # Since the papers used to inspire the augmentation did not mention these augmentations, I am disabling them
    mosaic=0.0,
    mixup=0.0,
    flipud=0.0,
    fliplr=0.0,
    translate=0.0,
    scale=0.0,
    perspective=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    copy_paste=0.0,
    erasing=0.0
)

In [ ]:
# Loading the package in each code section, so I run the following code brackets seperately
from ultralytics import YOLO
# Loading the best.pt weights from the YOLOv11 model above
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")

# Run validation on our test split and saving it in runs_YOLO11 in a folder called "val". The model uses the path in the dataset.yaml file to find the "test" split 
validation_results = v11_model.val(
    data="tiled_dataset/dataset.yaml",
    split="test",
    project="runs_YOLO11",
    name="val",
    exist_ok=True)

# The following section is to test how the model performs on the unlabeled images. The code is a bit crude, but I set it up like this so I could check any of the images without running the rest of the code. After each instance I have to compute the average confidence before running inference on the next image, since the results variable is overwritten every time.

In [ ]:
# Loading the package in each code section, so I run the following code brackets seperately
from ultralytics import YOLO
# Loading the best.pt weights from the YOLOv11 model above
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")
# Use the this fine-tuned model to run inference on our first unlabeled image
results = v11_model.predict(source="test_image1.jpg", imgsz=2000)
# Make the image pop up, with its class predictions and confidence scores
results[0].show()

In [ ]:
from ultralytics import YOLO
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")
results = v11_model.predict(source="test_image2.jpg", imgsz=2000)

results[0].show()

In [ ]:
from ultralytics import YOLO
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")
results = v11_model.predict(source="test_image3.jpg", imgsz=2000)

results[0].show()

In [ ]:
from ultralytics import YOLO
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")
results = v11_model.predict(source="test_image4.jpg", imgsz=2000)

results[0].show()

In [ ]:
from ultralytics import YOLO
v11_model = YOLO("runs_YOLO11/train/weights/best.pt")
results = v11_model.predict(source="test_image5.jpg", imgsz=2000)

results[0].show()

In [ ]:
from collections import defaultdict
# Create an empty dictionary of where every key has an empty list as default value, since we want to sum all confidence scores across each class
class_confidences = defaultdict(list)

# Loop through each of the bounding boxes in the results variable 
for box in results_new[0].boxes:

    # Save the class id associated with the bbox (as an integer, since it is 0 or 1)
    cls_id = int(box.cls[0])
    # Save the confidence associated with the bbox (as a float, since it is given as e.g. 0.81)
    conf = float(box.conf[0])
    # For every bounding box, append the associated confidence score. We end up with two keys, and each has an associated list of all the confidence scores for that class in the image
    class_confidences[cls_id].append(conf)

# In the dictionary loop through each key (class) one at a time:
for cls_id, confs in class_confidences.items():
    # Access the list in the current key and sum its values, which is divided by the total number of confidence scores in the list providing an average score
    avg_conf = sum(confs) / len(confs)
    # Print the average confidence score for each class/key rounding to 3 decimals 
    print(f"Class: {cls_id}, Average Confidence: {avg_conf:.3f}")